# Traditional sequence tagging

<sup>This notebook is a part of Natural Language Processing class at the University of Ljubljana, Faculty for computer and information science. Please contact [slavko.zitnik@fri.uni-lj.si](mailto:slavko.zitnik@fri.uni-lj.si) for any comments.</sub>

In machine-learning we can divide the classification models into two groups:

* Joint (generative) models - calculate probabilities over both observed and hidden data.
    * A model gives the probabilities P(c,d) and tries to maximize . 
    * Examples of such models are: naive Bayes, hidden Markov models, n-gram models.
* Discriminative (conditional) models - take the data as given and calculate probability over hidden structure (P(c|d)). Examples of such models are:
    * logistic regression, conditional (or maximum entropy) models, conditional random fields
    * also SVM, averaged perceptron, ... (but not directly probabilistic)
    
<img src="generativeVsDiscriminative.png" width="400px" />

In NLP, IR and speech processing we mostly use the conditional ones because:
* they will achieve better performance,
* they make it easy to incorporate lots of features,
* they allow for building language independent and reusable components.

Conditional models work well. Even with the same features as a joint model, a conditional estimation increases performance. An illustrative example on the task of word sense disambiguation (Klein and Manning, 2002):

| Objective     | Train data (CA)| Test data (CA) |
| ------------- |:--------------:|:--------------:|
| Joint         | 86.8           | 73.6           |
| Conditional   | **98.5**       | **76.1**       |

For those who are more interested and would like to better understand the ideas behind sequence tagging techniques, I propose to watch videos by Jurafsky and Manning that were used for their NLP Coursera course (relevant for this topic are sections 8,9,11 and 12).

## Hidden Markov Models

The often cited source source of Hidden Markov Models is Rabiner's paper *A tutorial on Hidden Markov Models and Selected Applications in Speech Recognition* (1989).

An HMM is specified by the following components:
* a set of states $Q$ (along with special start and end state $q_0,q_{end}$ that are not associated with observation)

$$
Q=q_1q_2\ldots q_N
$$

* a set of observations $O$

$$
O=o_1o_2\ldots o_N
$$

* a transition probability matrix $A$ (each $a_{ij}$ represents the probability of moving from state $i$ to state $j$, $\sum_{j=1}^{n} a_{ij} = 1\;\;\forall i$)

$$
A = 
\begin{bmatrix}
    a_{01} & a_{02} & a_{03} & \dots  & a_{0n} \\
    a_{11} & a_{12} & a_{13} & \dots  & a_{1n} \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    a_{n1} & a_{n2} & a_{n3} & \dots  & a_{nn}
\end{bmatrix}
$$

* a set of observation likelihoods (i.e., emission probabilities that express the probability of an observation $o_t$ being generated from state $i$)

$$
B=b_i(o_t)
$$

The algorithm is used to predict sequences of data using two simplifying assumptions. If we focus on first-order Markov chain, then the probability of a particular state is dependent only on the previous state:

$$
P(q_i|q_1\ldots q_{i-1}) = P(q_i|q_{i-1})
$$

Second, the probability of an output observation $o_i$ is dependent only on the state that produced observation $q_i$:

$$
P(o_i|q_1\ldots q_i,\ldots q_n,o_1,\ldots o_i,\ldots o_n) = P(o_i|q_i)
$$

A specific tag in a sequence is calculated as follows:

$$
\hat{T} = \textrm{argmax}_T\; \prod_i P(\textrm{word}_i|\textrm{tag}_i) \prod_i P(\textrm{tag}_i|\textrm{tag}_{i-1})
$$

Therefore, the probability of the whole sequence of states is calculated as follows:

$$
P(Q|O) = \prod_{i=1}^n P(o_i|q_i) \times \prod_{i=1}^n P(q_i|q_{i-1})
$$

To enable solving tagging problems using an HMM, three tasks needs to be implemented:
* Computing likelihood (the forwards algorithm): Given an HMM $\lambda=(A,B)$ and an observation sequence $O$, determine the likelihood $P(O|\lambda)$.
* Decoding (the viterbi algorithm): Given an observation sequence O and an HMM $\lambda=(A,B)$, discover the best hidden state sequence $Q$.
* Learning (the forward-backward or Baum-Welch algorithm): Given an observation sequence $O$ and the set of states in the HMM, learn the HMM parameters $A$ and $B$. 

### An HMM example

In [1]:
import nltk
nltk.download("treebank")

[nltk_data] Downloading package treebank to
[nltk_data]     /Users/slavkoz/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

In [3]:
from nltk.corpus import treebank
from nltk.tag import hmm

# Training data
train_data = treebank.tagged_sents()[:3000]
test_data = treebank.tagged_sents()[3000:6000]

print("An example of training data:\n\t{}\n".format(train_data[0]))

# Setup a trainer with default parameters and train a tagger
trainer = hmm.HiddenMarkovModelTrainer()
tagger = trainer.train_supervised(train_data)

# Print some tagger properties
print("Tagger info:\n\t{}\n".format(tagger))
print("Tagger symbols (first 100):\n\t{}\n".format(tagger._symbols[:100]))
print("Tagger states:\n\t{}\n".format(tagger._states))

print("\nClassification accuracy on TRAINING DATA: {:.2}\n".format(tagger.accuracy(train_data)))

print(f"\nTest data size: {len(test_data)}")
print("Classification accuracy on TEST DATA (our model): {:.2}".format(tagger.accuracy(test_data)))
print("Classification accuracy on TEST DATA (NLTK tagger): {:.2}\n".format(nltk.tag._get_tagger("eng").accuracy(test_data)))

# Tag new sentences
example = [word for word, pos in train_data[0]]
print("\nExample 0:\n\tOur tagger: {}\n\tNLTK tagger: {}".format(tagger.tag(example), nltk.pos_tag(example)))

example = "Today is a good day .".split()
print("\nExample 1:\n\tOur tagger: {}\n\tNLTK tagger: {}".format(tagger.tag(example), nltk.pos_tag(example)))

example = "We are going to master the Natural Language Processing Tools .".split()
print("\nExample 2:\n\tOur tagger: {}\n\tNLTK tagger: {}".format(tagger.tag(example), nltk.pos_tag(example)))


An example of training data:
	[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')]

Tagger info:
	<HiddenMarkovModelTagger 46 states and 10779 output symbols>

Tagger symbols (first 100):
	['Pierre', 'Vinken', ',', '61', 'years', 'old', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'Nov.', '29', '.', 'Mr.', 'is', 'chairman', 'of', 'Elsevier', 'N.V.', 'Dutch', 'publishing', 'group', 'Rudolph', 'Agnew', '55', 'and', 'former', 'Consolidated', 'Gold', 'Fields', 'PLC', 'was', 'named', '*-1', 'this', 'British', 'industrial', 'conglomerate', 'A', 'form', 'asbestos', 'once', 'used', '*', 'to', 'make', 'Kent', 'cigarette', 'filters', 'has', 'caused', 'high', 'percentage', 'cancer', 'deaths', 'among', 'workers', 'exposed', 'it', 'more', 'th

## Conditional (or Maximum Entropy) Markov Models

Let's start first with the maximum entropy classifier (MaxEnt), which is basically a multinomial logistic regression classifier. The classifier belongs to a group of exponential or log-linear classifiers. MaxEnt works by extracting some set of features from the input, combining them linearly (t.i., multiply each by a weight and then add them up), and then using this sum as an exponent.

A feature for tagging might be *"this word ends in -ing"* or *"the previous word was Mr."*. For each feature $f_i$, we need to have some weight $w_i$:

$$
p(c|x) = \frac{e^{\sum_iw_if_i}}{Z} = \frac{\mathrm{exp}(\sum_iw_if_i)}{Z}
$$

Let's say we have $N$ features and $C$ classes $\{c_1,c_2\ldots c_C\}$:

$$
p(c|x) = \frac{\mathrm{exp}(\sum_{i=0}^{N}w_{ci}f_i(c,x))} {\sum_{c'\in C}\mathrm{exp}(\sum_{i=0}^{N}w_{c'i}f_i(c',x))}
$$

We are still doing the standard classification, so $x$ may be one token (e.g., word) in a sequence. Now we need to define some feature functions, for example:

$$
f_1(c,x) =  
\begin{cases} 
      1 & \textrm{if}\;x=\textrm{"Mirko"}\;\&\;c = \textrm{ PER} \\
      0 & \textrm{otherwise}
\end{cases}
$$

$$
f_2(c,x) =  
\begin{cases} 
      1 & \textrm{if POS tag of }\;x=\textrm{"Noun"} \\
      0 & \textrm{otherwise}
\end{cases}
$$

$$
f_3(c,x) =  
\begin{cases} 
      1 & \textrm{if word }\;x\;\textrm{ starts with upper case} \\
      0 & \textrm{otherwise}
\end{cases}
$$

$$
f_4(c,x) =  
\begin{cases} 
      1 & \textrm{if word before }\;x=\textrm{"Mr."}\;\&\;c = \textrm{ PER} \\
      0 & \textrm{otherwise}
\end{cases}
$$

$$
f_5(c,x) =  
\begin{cases} 
      1 & \textrm{if word before }\;x=\textrm{"Mr."}\;\&\;c = \textrm{ ORG} \\
      0 & \textrm{otherwise}
\end{cases}
$$

$$
f_6(c,x) =  
\begin{cases} 
      1 & \textrm{if word }\;x=\textrm{ appears in the list of organizations}\;\&\;c = \textrm{ ORG} \\
      0 & \textrm{otherwise}
\end{cases}
$$

Now we would train the model and get weights for all of the features regarding the classes. Let's have an example sentence "Dr. Mirko and Mr. Slavko are working at Fušarija Inc.". Now we are interested in the word "Slavko", so according to it, we can populate the following table (note, we still use token-level MaxEnt):

$$
\begin{array}{lcccccc}
 &1&2&3&4&5&6 \\ \hline
f_i(\textrm{"ORG"}, \textrm{"Slavko"})&0&1&1&0&1&0 \\ \hline
w_{\textrm{"ORG"}i}&&.9&1.1&&-.2& \\ \hline
f_i(\textrm{"PER"}, \textrm{"Slavko"})&0&1&1&1&0&0 \\ \hline
w_{\textrm{"PER"}i}&&.7&.9&.95&& \\ \hline
\end{array}
$$

According to the above, we can calculate the following probabilities:

$$
P(\textrm{"ORG"}|\textrm{"Slavko"}) = \frac{e^{.9}e^{1.1}e^{-.2}}{e^{.9}e^{1.1}e^{-.2}+e^{.7}e^{.9}e^{.95}} \approx \frac{6.05}{18.86} \approx 0.32
$$

$$
P(\textrm{"PER"}|\textrm{"Slavko"}) = \frac{e^{.7}e^{.9}e^{.95}}{e^{.9}e^{1.1}e^{-.2}+e^{.7}e^{.9}e^{.95}} \approx \frac{12.81}{18.86} \approx 0.68
$$

Now we can perform classification using formula 

$$
\hat{c} = \textrm{argmax}_{c\in C}\;P(c|x),
$$

which results that the word "Slavko" in the above sequence is classified as a person (i.e. *PER*).

Now we want a such classifier to predict sequences of classes and not classes of tokens separately. Using a combination of MaxEnt and HMM, we can achieve this using MaxEnt and VIterbi. As we know, Maximum Entropy Models compute $P(T|W)$ directly in one model and does not rely on a Bayes rule ($P(W|T)P(W)$), as it is discriminative model rather than generative. A specific tag in a sequence is calculated as follows:

$$
\hat{T} = \textrm{argmax}_T\;\prod_i P(\textrm{tag}_i|\textrm{word}_i,\textrm{tag}_{i-1})
$$

Therefore, the probability of a sequence of states is calculated as follows:

$$
P(Q|O)=\prod_{i=1}^n P(q_i|q_{i-1},o_i)
$$

An example of using the NLTK's MaxentClassifier along with Viterbi to achieve the functionality of MEMMs is available in the following repository: [https://github.com/yh1008/MEMM](https://github.com/yh1008/MEMM).

## Conditional Random Fields

CRFs is another sequence model, but before we present it, let's look back to HMMs and MEMMs.

CRFs and MEMMS are discriminative sequence models whereas HMMs are generative sequence models. HMM essentially uses Bayes Rule as a local model over the transition and emission probabilities, whereas CRF and MEMM's local models are MaxEnt models over transition and observable features. 

The biggest disadvantage of HMMs is that it is hard to represent the data in a form to use many features. MEMMs solves this by using many features but still remains locally normalized, which results in [a label bias problem](https://awni.github.io/label-bias/). The CRFs solves the problem by globaly normalizing against the whole sequence. CRFs is therefore widely used and applied and gives state-the-art results in many domains.

For more info, see a nice presentation in *From_HMM_to_CRF.ppt*.

The CRF model is represented as

$$
P(c|d,\lambda) = \frac{\textrm{exp}\sum_i\lambda_if_i(c,d)}{\sum_{c'}\textrm{exp}\sum_i\lambda_if_i(c',d)},
$$

where the space of $c'$s is now the space of all possible sequences.

### A CRF example

NLTK library does not include a CRF implementation but it provides an interface to a widely and efficient C++ implementation [CRFSuite](http://www.chokkan.org/software/crfsuite/). 

#### How to use a CRF

```python
##################
### PSEUDOCODE ###
##################

# 1. Define feature functions
def getFeatures(tokens, idx):
    token = tokens[idx]

    feature_list = []

    # Capitalization
    if containtsCapsWord(token):
        feature_list.append('CAPS_WORD')

    # Number
    if sum([re.search(num_pattern, token) is not None for token in tokens]) > 0:
        feature_list.append('HAS_NUM')

    if textUniqueLength(tokens) < 5:
        feature_list.append('SHORT')

    if containsOneOf_5w1h(token) == 1:
        feature_list.append('5W1H')

    if hasQuestionmark(token) == 1:
        feature_list.append("QUESTION")

    if hasThank(token) == 1:
        feature_list.append("THANK")

    if hasExclamationMark(token) == 1:
        feature_list.append("EXCLAMATION")

    if hasPositiveFeedback(token) == 1:
        feature_list.append("POS_FEED")

    if hasDuplicateWords(token) == 1:
        feature_list.append("DUPLIC")

    feature_list.append("SENT=" + str(sentimentValue(token)))

    # TOP TFIDF LEMMAS
    for (lemma, _) in topNtfidfLemmas(token, 5):
        feature_list.append("LEMMA=" + lemma)

    if hasLink(token) == 1:
        feature_list.append("LINK")

    if hasPunctuations(token)[":"] == 1:
        feature_list.append("COLON")

    if hasPunctuations(token)[".."] == 1:
        feature_list.append("DOTS")

    if numberOfEmoticons(token) > 0:
        feature_list.append("EMOTICONS")

    if numberOfUnicodeEmoticons(token) > 0:
        feature_list.append("U_EMOTICONS")

    if numberOfInterjections(token) > 0:
        feature_list.append("INTERJECTIONS")

    if verbPos(token) == -1:
        feature_list.append("NO_VERB")
    elif verbPos(token) == 0:
        feature_list.append("VERB_FIRST")
    elif verbPos(token) == len(token["tokens"]) - 1:
        feature_list.append("VERB_LAST")
    else:
        feature_list.append("VERB_MIDDLE")

    for (modal, f) in zip(modals, modalsFunctionGenerator()):
        if f(token) > 0:
            feature_list.append("MODAL=" + modal)

    if numberOfNegations(token) > 0:
        feature_list.append("NEG")

    if hasFutureVerb(token) == 1:
        feature_list.append("FUTURE")

    if verbTense(token)["hasPastV"] == 1:
        feature_list.append("TENSE=hasPastV")

    if verbTense(token)["hasIngV"] == 1:
        feature_list.append("TENSE=hasIngV")

    if verbTense(token)["hasImperativeV"] == 1:
        feature_list.append("TENSE=hasImperativeV")

    if pronominalCues(token)["has_1stperson_sg"] > 0:
        feature_list.append("PRONOMINAL=has_1stperson_sg")

    if pronominalCues(token)["has_1stperson_pl"] > 0:
        feature_list.append("PRONOMINAL=has_1stperson_pl")

    if pronominalCues(token)["has_2ndperson"] > 0:
        feature_list.append("PRONOMINAL=has_2ndperson")

    if pronominalCues(token)["has_3rdperson"] > 0:
        feature_list.append("PRONOMINAL=has_3rdperson")

    return feature_list

# 2. Prepare data
data = []
for post in getAllPosts(examples):
    data.append([(example, str((example['class']))) for example in post])

# 3. Prepare data for training (and testing if needed)
random.shuffle(data)
n_test_samples = int(0.3 * len(data))
train_data = data[:-n_test_samples]
test_data = data[-n_test_samples:]

# 4. Train a classifier (built classifier will be stored into a file "model.crf.tagger")
ct = CRFTagger(feature_func=getFeatures, verbose=True, training_opt={})
ct.train(train_data, "model.crf.tagger")

# 5. Evaluate classifier
print(ct.evaluate(test_data))

# 6. Tag new sequences
print(ct.tag(test_data[0]))

# When we have already trained a model and would just like to tag new data
ct = CRFTagger(feature_func=getFeatures, verbose=True, training_opt={})
ct.set_model_file("model.crf.tagger")
print(ct.tag(test_data[0])`
```

#### Simple CRF for part-of-speech tagging

What are the feature functions used in the example below?

In [3]:
import nltk
from nltk.corpus import treebank
from nltk.tag import crf

# Training data
train_data = treebank.tagged_sents()[:3000]
test_data = treebank.tagged_sents()[3000:6000]
print("An example of training data:\n\t{}\n".format(train_data[0]))

# Setup a trainer with default parameters and train a tagger
tagger = crf.CRFTagger()
tagger.train(train_data, "model.crf.tagger")

print("\nClassification accuracy on TRAINING DATA: {:.2}\n".format(tagger.accuracy(train_data)))

print(f"\nTest data size: {len(test_data)}")
print("Classification accuracy on TEST DATA (our model): {:.2}".format(tagger.accuracy(test_data)))

# Tag new sentences
example = [word for word, pos in train_data[0]]
print("\nExample 0:\n\tOur tagger: {}\n\tNLTK tagger: {}".format(tagger.tag(example), nltk.pos_tag(example)))

example = "Today is a good day .".split()
print("\nExample 1:\n\tOur tagger: {}\n\tNLTK tagger: {}".format(tagger.tag(example), nltk.pos_tag(example)))

example = "We are going to master the Natural Language Processing Tools .".split()
print("\nExample 2:\n\tOur tagger: {}\n\tNLTK tagger: {}".format(tagger.tag(example), nltk.pos_tag(example)))

# Where are the feature functions here?

An example of training data:
	[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')]


Classification accuracy on TRAINING DATA: 0.96


Test data size: 914
Classification accuracy on TEST DATA (our model): 0.95

Example 0:
	Our tagger: [('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')]
	NLTK tagger: [('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('di

The example above shows the use of NLTK's CRFSuite wrapper - [python-crfsuite library](https://python-crfsuite.readthedocs.io/en/latest/). In the library's documentation you can find an example of how to use the library directly - [jupyter notebook](https://github.com/scrapinghub/python-crfsuite/blob/master/examples/CoNLL%202002.ipynb).

*NOTE:* If you get an error '*NameError: name 'pycrfsuite' is not defined*,' remove try/except block around the pycrfsuite import in the '.venv/lib/python3.10/site-packages/nltk/tag/crf.py'. After that reset kernel and re-run. More info: [https://github.com/nltk/nltk/issues/3019](https://github.com/nltk/nltk/issues/3019).

## Exercises

Try to do the following on your own:
* Implement your own HMM classifier based on the Rabiner's tutorial.
* Adjust the number of training sequences for the HMM POS tagger and observe differences.
* Try some CRF parameters - e.g., verbose, feature_func, training_opt.
* Try to implement named entity recognition on the ssj500k dataset for slovene (according to the paper [Razpoznavanje imenskih entitet v slovenskem jeziku, Štajner in sod. (2013)](http://revije.ff.uni-lj.si/slovenscina2/article/download/6926/6620)).
* Find your own sequence tagging problem and try it - for example, see CoNLL tasks.